# つくりながら学ぶ！深層強化学習 PyTorchによる実践プログラミング

## 迷路問題

前回までで、行動反復法と価値反復法による強化学習アルゴリズムを実装してきた

今回は、これらのアルゴリズムを選択して強化学習できるようにフレームワークを導入して汎用化してみる

In [6]:
using ReinforcementLearning, NaNStatistics, Distributions

"エージェントの行動: 上、右、下、左移動"
@enum MazeAction UP=1 RIGHT DOWN LEFT

"迷路問題の状態: S0(START), S1, ..., S8(GOAL)"
@enum MazeState S0=1 S1 S2 S3 S4 S5 S6 S7 S8

"迷路問題環境モデル"
Base.@kwdef mutable struct MazeEnv <: AbstractEnv
    state::Int = Int(S0) # 現在の状態 = エージェント位置
    reward::Union{Nothing, <:Number} = nothing # 報酬
    state_action_history::Vector{NamedTuple} = [] # エージェントの行動記録
end

"状態空間: 環境モデル内のすべての状態"
RLBase.state_space(env::MazeEnv) ::Tuple = Int.(instances(MazeState))

"行動空間: エージェントがとりうるすべての行動"
RLBase.action_space(env::MazeEnv) ::Tuple = Int.(instances(MazeAction))

"現在の状態"
RLBase.state(env::MazeEnv) ::Int = env.state

"報酬: エージェントがゴールしない限り報酬は 0"
RLBase.reward(env::MazeEnv) ::Int = isnothing(env.reward) ? 0 : env.reward

"終了条件: 報酬が得られたとき（エージェントがゴールしたとき）"
RLBase.is_terminated(env::MazeEnv) ::Bool = !isnothing(env.reward)

"環境モデル初期化"
RLBase.reset!(env::MazeEnv) = begin
    env.state = Int(S0)
    env.reward = nothing
end

"1step の実行処理"
(env::MazeEnv)(action::Int) = begin
    # 行動記録
    push!(env.state_action_history, (state = state(env), action = action))

    # 状態遷移表
    state_map = Dict(
        Int(UP)    => -3,
        Int(RIGHT) => +1,
        Int(DOWN)  => +3,
        Int(LEFT)  => -1,
    )
    env.state += state_map[action]

    # ゴールしたとき報酬 1 を与える
    if env.state === Int(S8)
        push!(env.state_action_history, (state = Int(S8), action = NaN))
        env.reward = 1
    end
end

env = MazeEnv()

# MazeEnv

## Traits

| Trait Type        |                  Value |
|:----------------- | ----------------------:|
| NumAgentStyle     |          SingleAgent() |
| DynamicStyle      |           Sequential() |
| InformationStyle  | ImperfectInformation() |
| ChanceStyle       |           Stochastic() |
| RewardStyle       |           StepReward() |
| UtilityStyle      |           GeneralSum() |
| ActionStyle       |     MinimalActionSet() |
| StateStyle        |     Observation{Any}() |
| DefaultStateStyle |     Observation{Any}() |

## Is Environment Terminated?

No

## State Space

`(1, 2, 3, 4, 5, 6, 7, 8, 9)`

## Action Space

`(1, 2, 3, 4)`

## Current State

```
1
```


In [ ]:
"""
    pi_theta(theta::Matrix{Number}) = pi::Matrix{Number} (8x4)

方策パラメータ θ を行動方策 π に変換する関数

- 行動の重み θ から、行動の採用確率 π_θ に変換する
- 例: S0 [NaN 1.0 1.0 NaN] の場合
    - 行動の採用確率は [0.0 0.5 0.5 0.0] となる
"""
pi_theta(theta::Matrix) =
    # 各値をその行での割合 (値 / その行の合計値) に変換
    ## NaN 値を無視して合計を出すには NaNStatistics.nansum を使うと良い
    theta ./ nansum(theta, dims = 2) |>
        # NaN 値を 0.0 に変換する
        theta -> map(theta) do t isnan(t) ? 0.0 : t end


"""
    pi_theta_softmax(theta::Matrix{Union{Nothing, Number}}) = pi::Matrix{Number} (8x4)

方策パラメータ θ を行動方策 π に変換する Softmax 関数
"""
pi_theta_softmax(theta::Matrix) = begin
    # 逆温度β: 小さいほど行動がランダムになりやすい
    beta = 1.0

    # exp(βθ)::Matrix{Number} (8x4): ここでマイナス値も正規化される
    exp_theta = exp.(beta .* theta)

    # π_softmax(θ)
    ## 欠損値を無視して行ごとの列値合計を算出するために nansum(::Matrix, dims=2) を使う
    pi = exp_theta ./ nansum(exp_theta, dims=2)

    # 欠損値を 0.0 に変換して完了
    map(pi) do v isnan(v) ? 0.0 : v end
end

"方策 π(θ): 方策勾配法"
mutable struct MazePGPolicy <: AbstractPolicy
    pi::Matrix{<:Number}
end

"現在の状態でエージェントがとる行動を決める方策"
(pi::MazePolicy)(env::MazeEnv) ::Int = wsample(action_space(env), pi.pi[state(env), :])


""